In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from random import randrange
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import log_loss
from datetime import datetime
import autosklearn.classification
from sklearn.metrics import ConfusionMatrixDisplay
from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import sys
sys.path.append('/home/krzysiek/drivendata/drivendata_competitions/functions')
from eda_automations import *
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')

In [3]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
labels_data = pd.read_csv('https://github.com/KCristopher/drivendata_competitions/raw/main/data/training_set_labels.csv')

In [5]:
labels_data.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


The data contains the info about waterpoints in Tanzania. Each observations correspond to some waterpoint and is described in terms of the variables such as the height at which the waterpoint is located, type of source of the water (for example rain or spring) or the institution which built the well.  

We hope that we would be able to discern which waterponits are functional, which are not, and which needs to be repaired by examining the levels of all these different variables.

In [6]:
labels_data.index = labels_data['id']
del labels_data['id']

In [7]:
labels_data.head(3)

,status_group
id,
69572,functional
8776,functional
34310,functional


In [8]:
y = labels_data.copy()
X = df.copy()

Splitting the data into training and testing sets :

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Exploring training data

<center>Checking the general structure of the data, data types and  missing values :</center>

In [10]:
summarize_df(X_train)


Glance at the data : 



,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3607,454,50.0,2013-02-27,Dmdd,2092,DMDD,35.426020,-4.227446,Narmo,0,...,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
20413,14146,0.0,2011-07-10,Kkkt,0,KKKT,32.499866,-9.081222,Mahakama,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
52806,47410,0.0,2011-04-12,NaN,0,NaN,34.060484,-8.830208,Shule Ya Msingi Chosi A,0,...,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
50091,1288,300.0,2011-04-05,Ki,1023,Ki,37.032690,-6.040787,Kwa Mjowe,0,...,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,other,other



Data types, dimension of the data : 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     47520 non-null  int64  
 1   amount_tsh             47520 non-null  float64
 2   date_recorded          47520 non-null  object 
 3   funder                 44644 non-null  object 
 4   gps_height             47520 non-null  int64  
 5   installer              44631 non-null  object 
 6   longitude              47520 non-null  float64
 7   latitude               47520 non-null  float64
 8   wpt_name               47520 non-null  object 
 9   num_private            47520 non-null  int64  
 10  basin                  47520 non-null  object 
 11  subvillage             47224 non-null  object 
 12  region                 47520 non-null  object 
 13  region_code            47520 non-null  int64  
 14  district_cod

None


Missing values across variables : 



scheme_name              22523
scheme_management         3102
installer                 2889
funder                    2876
public_meeting            2689
permit                    2439
subvillage                 296
management_group             0
waterpoint_type              0
extraction_type              0
extraction_type_group        0
source_class                 0
extraction_type_class        0
management                   0
source_type                  0
source                       0
payment_type                 0
water_quality                0
construction_year            0
quality_group                0
quantity                     0
quantity_group               0
payment                      0
id                           0
amount_tsh                   0
recorded_by                  0
population                   0
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
basin   


Missing values - % of the data : 


scheme_name              47.40
scheme_management         6.53
installer                 6.08
funder                    6.05
public_meeting            5.66
permit                    5.13
subvillage                0.62
management_group          0.00
waterpoint_type           0.00
extraction_type           0.00
extraction_type_group     0.00
source_class              0.00
extraction_type_class     0.00
management                0.00
source_type               0.00
source                    0.00
payment_type              0.00
water_quality             0.00
construction_year         0.00
quality_group             0.00
quantity                  0.00
quantity_group            0.00
payment                   0.00
id                        0.00
amount_tsh                0.00
recorded_by               0.00
population                0.00
ward                      0.00
lga                       0.00
district_code             0.00
region_code               0.00
region                    0.00
basin   

In [11]:
codes = [c for c in X_train.columns if 'code' in c]
codes

['region_code', 'district_code']

<center>Getting numeric summaries of the data</center>

In [12]:
summarize_numeric_data(X_train, exclude = ['id'] + codes)


The correlation between the variables in the dataset :



,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year
amount_tsh,1.00,0.07,0.02,-0.05,0.00,0.01,0.06
gps_height,0.07,1.00,0.15,-0.04,0.01,0.13,0.66
longitude,0.02,0.15,1.00,-0.43,0.02,0.09,0.40
latitude,-0.05,-0.04,-0.43,1.00,0.01,-0.02,-0.25
num_private,0.00,0.01,0.02,0.01,1.00,0.00,0.03
population,0.01,0.13,0.09,-0.02,0.00,1.00,0.26
construction_year,0.06,0.66,0.40,-0.25,0.03,0.26,1.00



Descriptive statistics of numeric data : 



,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year
count,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00
mean,322.05,668.75,34.09,-5.71,0.50,179.53,1303.35
std,3200.62,692.97,6.54,2.94,13.25,472.77,950.76
min,0.00,-63.00,0.00,-11.65,0.00,0.00,0.00
25%,0.00,0.00,33.08,-8.53,0.00,0.00,0.00
50%,0.00,370.00,34.91,-5.02,0.00,25.00,1986.00
75%,20.00,1320.00,37.18,-3.33,0.00,213.00,2004.00
max,350000.00,2770.00,40.35,-0.00,1776.00,30500.00,2013.00


In [13]:
look_at_variables_values(X_train)


Unique values from id, limited to 30

17336    24344
15468    14218
32413    48749
27565    70267
47260    12539
12429    12172
46156    21636
36722    45373
4853      2703
43873     4902
44417    37025
46197    43243
1823     67090
14034     8206
39076    34323
20730    58851
5377     27389
8354     19298
43324    13613
32890    45907
15429    34583
15812    11019
24980    58404
46270    65603
23733    48734
35471    36280
9956     30038
6538     63187
16215    40011
13122    37547
dtype: int64


Unique values from amount_tsh, limited to 30

26       400.0
33       700.0
62     30000.0
19        20.0
17     10000.0
8       1000.0
89      5400.0
57     70000.0
6        500.0
42      7200.0
37     12000.0
63      4500.0
75     60000.0
67       350.0
59      6500.0
61     11000.0
21       200.0
92    250000.0
22       600.0
24      5000.0
91       900.0
38     25000.0
95      1400.0
13      1500.0
78      5500.0
29         6.0
90    100000.0
55        60.0
12        40.0
2        300.0


0    communal standpipe
4         cattle trough
5                   dam
1             hand pump
3       improved spring
2                 other
dtype: object



We can see that there is one entry of 0 in the column construction_year.

In [14]:
X_train[X_train.construction_year == 0].shape

(16503, 40)

In [15]:
X_train.columns.tolist()

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [16]:
cols_subset = ['ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality']

In [17]:
X_train[X_train.construction_year == 0][cols_subset].sample(n = 100).head(50)

,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality
55132,Karitu,0,False,GeoData Consultants Ltd,VWC,NaN,False,0,other,other,other,vwc,user-group,never pay,never pay,soft
38877,Ikolo,0,True,GeoData Consultants Ltd,VWC,Ngana water supplied scheme,True,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft
50801,Sumve,0,True,GeoData Consultants Ltd,VWC,JUWAMASU,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft
33382,Badi,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,swn 80,swn 80,handpump,wug,user-group,other,other,soft
25847,Chipanga,0,True,GeoData Consultants Ltd,VWC,Chip,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft
7852,Nyabusozi,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft
39859,Mwakilyambiti,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft
48701,Bunamhala,0,NaN,GeoData Consultants Ltd,WUG,NaN,False,0,other,other,other,wug,user-group,unknown,unknown,unknown
2598,Ikindwa,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,pay when scheme fails,on failure,soft
37845,Kiloleli,0,True,GeoData Consultants Ltd,VWC,Mradi wa maji wa Kiloleli,True,0,other,other,other,vwc,user-group,never pay,never pay,soft


In [53]:
def count_values(df, value):
    """
    Returns a pandas DataFrame with the names of the original features in the index and the number of
    occurrences of the value as the values of the first column and the same as the percentage of all
    observations in the second column.

    If the passed value is boolean, compares with boolean values in the DataFrame. If the passed value
    is a string, compares with object (string) values in the DataFrame. Otherwise, only compares with
    values of the same data type.

    Parameters:
    -----------
    df: pandas DataFrame.

    value : str / int / float / bool.
        The value to be counted.

    Returns:
    --------
    pandas DataFrame.
    """

    if isinstance(value, bool):
        value_mask = df.apply(lambda x: isinstance(x[0], bool), axis=0)
    elif isinstance(value, str):
        value_mask = df.dtypes == 'object'
    elif isinstance(value, int):
        value_mask = df.dtypes == 'int64'
    elif isinstance(value, float):
        value_mask = df.dtypes == 'float64'
    else:
        raise TypeError("Input value must be of type bool, str, int, or float.")

    count_series = df.loc[:, value_mask].eq(value).sum().sort_values(ascending=False)
    percent_series = count_series / len(df) * 100
    percent_series = percent_series.round(1)

    data = {
        f"Count of {value}": count_series,
        f"{value}'s as % of Total": percent_series
    }
    result_df = pd.DataFrame(data)
    result_df.index.name = 'Feature'
    result_df.sort_values(by=f"Count of {value}", ascending=False, inplace=True)

    return result_df[result_df['Count of {}'.format(value)] > 0]


In [47]:
X_train[['permit', 'scheme_management']]

,permit,scheme_management
3607,True,Water Board
50870,True,VWC
20413,False,VWC
52806,True,VWC
50091,True,VWC
...,...,...
54343,True,Water Board
38158,False,VWC
860,False,Private operator
15795,True,WUG


In [20]:
X_train.permit.iloc[0] == True

True

<center>Examining potentially not obvious missing values</center>

In [65]:
count_values(df = X_train, value = 0)

,Count of 0,0's as % of Total
Feature,,
num_private,46903,98.7
population,17048,35.9
construction_year,16503,34.7
gps_height,16275,34.2
district_code,19,0.0
id,1,0.0


In [67]:
count_values(df = X_train, value = '-')

,Count of -,-'s as % of Total
Feature,,
installer,3,0.0


In [79]:
X_train.num_private.value_counts(normalize = True).round(3).head(3)

0    0.987
6    0.001
1    0.001
Name: num_private, dtype: float64

Regardless whether num_private, for which description is not available, can really be 0 or it is a missing value, we can safely remove it, because the feature which has the same value in 99 % of cases wouldn't be very useful anyway. There is a big chance that t is a missing value, too.

In [23]:
X_train.shape

(47520, 40)

In [24]:
zero_values_per_feature = count_, values(X_train, 0)
zero_values_per_feature

NameError: name 'count_' is not defined

In [ ]:
for z in zero_values_per_feature.index :
    print()
    if len(X_train[z].unique()) < 30 :
        display(X_train[z].value_counts())
    print()

# This is wrong

In [ ]:
X_train[X_train.construction_year == 0].groupby('management').size().sort_values(ascending = False)\
.div(len(X_train[X_train.construction_year == 0])).round(2)

In [ ]:
for c in X_train[X_train.construction_year == 0].columns :
    if X_train[X_train.construction_year == 0][c].dtype == 'O' :
        if len(X_train[X_train.construction_year == 0][c].unique()) < 10 :
            print()
            print('The fraction of water points records without construction year grouped by {}.'.format(c))
            print()
            print(X_train[X_train.construction_year == 0].groupby(c).size()\
                  .sort_values(ascending = False).div(len(X_train[X_train.construction_year == 0])).round(2))
            print()

In [ ]:
perc_record_zero_year = round(len(df[df.construction_year == 0]) / X_train.shape[0] * 100, 1)

In [ ]:
print(f'As many as {perc_record_zero_year} % of records are missing the year info.')

In [ ]:
X_train.num_private.value_counts()